# LangChain LCEL 인터페이스

LCEL(LangChain Expression Language)은 LangChain에서 제공하는 표현식 언어로, 다양한 인터페이스를 통해 AI 모델과 상호작용할 수 있게 해줍니다. 이 노트북에서는 다음과 같은 LCEL의 주요 인터페이스를 살펴보겠습니다:

- [`stream`](#stream): 생성시 실시간 호출
- [`invoke`](#invoke): 입력에 대해 체인 호출
- [`batch`](#batch): 입력 목록에 대해 일괄처리후 호출
- [비동기 호출](#async-calls): 비동기 방식으로 호출하는 방법
- [병렬 실행](#parallel-execution): 여러 체인을 동시에 실행하는 방법

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 기본 체인 설정

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 모델 설정
model = ChatOpenAI(streaming=True)

# 템플릿 사용
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")

# 체인 구성 - 파이프라인 연산자(|)를 사용해 구성요소 연결
chain = prompt | model | StrOutputParser()

## Stream

`.stream()` 메서드를 사용하면 모델의 출력을 실시간으로 스트리밍할 수 있습니다. 토큰이 생성됨에 따라 바로 확인할 수 있어 대화형 애플리케이션에 유용합니다.

In [ ]:
# 스트리밍 사용 예시
for token in chain.stream({"topic": "ChatGPT"}):
    print(token, end="", flush=True)

## Invoke

`.invoke()` 메서드는 입력값에 대해 체인을 호출하고 결과를 반환합니다. 한 번의 호출로 완성된 결과를 얻을 때 유용합니다.

In [ ]:
# 일반적인 호출 방식
chain.invoke({"topic": "claudeAi"})

## Batch

`.batch()` 메서드는 여러 입력값을 한 번에 처리할 때 사용합니다. 이를 통해 여러 작업을 효율적으로 일괄 처리할 수 있습니다.

In [ ]:
# 여러 입력에 대한 일괄 처리
chain.batch([{"topic": "claudeAi"}, {"topic": "nvidia"}])

## 비동기 호출(Async Calls)

비동기 메소드는 프로그램이 특정 작업을 수행하고 있을 때, 그 작업이 완료될 때까지 프로그램 전체를 멈추지 않고 다른 작업을 계속 수행할 수 있게 해주는 방법입니다. LCEL에서는 `astream`, `ainvoke`, `abatch` 메서드를 제공합니다.

In [ ]:
# 비동기 스트리밍
async for token in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(token, end="", flush=True)

In [ ]:
# 비동기 호출
my_process = chain.ainvoke({"topic": "NVDA"})
# 작업이 완료될 때까지 기다립니다
await my_process

In [ ]:
# 비동기 일괄 처리
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)
# 작업이 완료될 때까지 기다립니다
await my_abatch_process

## 병렬 실행(Parallel Execution)

RunnableParallel은 여러 체인을 병렬로 실행할 수 있게 해주는 클래스입니다. 각 체인이 독립적으로 동작하므로 여러 작업을 동시에 수행할 때 유용합니다.

In [ ]:
from langchain_core.runnables import RunnableParallel

# 첫 번째 체인: 음식의 주 성분을 물어보는 체인
chain1 = (
    PromptTemplate.from_template("{food} 의 주 성분은 뭐야?")
    | model
    | StrOutputParser()
)

# 두 번째 체인: 요리 방법을 물어보는 체인
chain2 = (
    PromptTemplate.from_template("{cooking} 조리 방법이뭐야?")
    | model
    | StrOutputParser()
)

# 두 체인을 병렬로 실행하는 결합된 체인
combined = RunnableParallel(ingredients=chain1, cook=chain2)

In [ ]:
# 두 체인에 각각 다른 입력을 제공하여 병렬로 실행
combined.invoke([{"food": "닭가슴살"}, {"cooking": "봉골레 파스타"}])

In [ ]:
# 첫 번째 체인에 여러 입력을 일괄 처리
chain1.batch([{"food": "포도"}, {"food": "사과"}])

In [ ]:
# 두 번째 체인에 여러 입력을 일괄 처리
chain2.batch([{"cooking": "라면"}, {"cooking": "김치찌개"}])